In [14]:
import numpy as np

In [15]:
# Step1: 生成Young图 并且通过hook formula 计算出右cell大小

In [26]:
## Young tableaux
def insert_into_row(row, value):
    """
    Inserts a value into a row of the tableau using the bumping rule.
    Returns the bumped value if any, otherwise None.
    """
    for i, x in enumerate(row):
        if value < x:
            row[i] = value
            return x
    row.append(value)
    return None

def robinson_schensted(permutation):
    """
    Implements the Robinson–Schensted algorithm to generate the P and Q tableaux.
    """
    P = []  # The insertion tableau
    Q = []  # The recording tableau
    
    for step, value in enumerate(permutation, 1):
        row_num = 0
        to_insert = value
        
        while row_num < len(P):
            # print(len(P))
            bumped = insert_into_row(P[row_num], to_insert)
            if bumped is None:
                break
            to_insert = bumped
            row_num += 1
        
        if row_num == len(P):
            # Create a new row if we couldn't insert into any existing row
            P.append([to_insert])
        
        # Update the recording tableau
        if row_num == len(Q):
            Q.append([step])
        else:
            Q[row_num].append(step)
    
    return P, Q

def print_tableau(tableau):
    """
    Prints a tableau in a nicely formatted way.
    """
    for row in tableau:
        print(" ".join(map(str, row)))

# Example usage
permutation = [10,3,11,2,12,1,13,6,4,8,5,7,9]
P, Q = robinson_schensted(permutation)

print("P tableau:")
print_tableau(P)

print("\nQ tableau:")
print_tableau(Q)


P tableau:
1 4 5 7 9
2 6 8 13
3 11 12
10

Q tableau:
1 3 5 7 13
2 8 10 12
4 9 11
6


In [27]:
def get_shape(P):
    """
    Returns the shape of a Young tableau.
    """
    return tuple(len(row) for row in P)

get_shape(P)

(5, 4, 3, 1)

In [28]:
from math import factorial

def hook_lengths(shape):
    """
    Compute the hook lengths for each box in a Young diagram of given shape.
    Args:
        shape (list of int): A list where each element represents the number of boxes in a row.
    Returns:
        list of list: A matrix of the same shape where each entry is the hook length of the corresponding box.
    """
    rows = len(shape)
    hooks = []
    
    for i, row_length in enumerate(shape):
        hooks.append([])
        for j in range(row_length):
            # Calculate the hook length for the box at (i, j)
            down = sum(1 for k in range(i + 1, rows) if j < shape[k])  # Boxes below
            right = shape[i] - j - 1  # Boxes to the right
            hooks[-1].append(1 + down + right)  # Include the box itself
    
    return hooks

def hook_formula(shape):
    """
    Compute the number of standard Young tableaux for a given shape using the hook formula.
    Args:
        shape (list of int): A list where each element represents the number of boxes in a row.
    Returns:
        int: The number of standard Young tableaux for the given shape.
    """
    n = sum(shape)  # Total number of boxes
    hooks = hook_lengths(shape)
    
    # Compute the product of all hook lengths
    hook_product = 1
    for row in hooks:
        for h in row:
            hook_product *= h
    
    # Apply the hook formula
    return factorial(n) // hook_product

def print_hook_lengths(hooks):
    """
    Pretty-print the hook lengths matrix.
    """
    for row in hooks:
        print(" ".join(map(str, row)))

# Example usage
shape = get_shape(P)  # A Young diagram with 3 boxes in the first row, 2 in the second, and 1 in the third
hooks = hook_lengths(shape)
print("Hook lengths:")
print_hook_lengths(hooks)

num_tableaux = hook_formula(shape)
print(f"\nNumber of standard Young tableaux for shape {shape}: {num_tableaux}")



Hook lengths:
8 6 5 3 1
6 4 3 1
4 2 1
1

Number of standard Young tableaux for shape (5, 4, 3, 1): 15015


In [29]:
# Step2: 初始化
S = set()
N = set()

In [30]:
import itertools

def extract_subpermutations(perm, length=4):
    """
    Extract all subsequences of length 4 where the indices satisfy i < j < k < l.
    """
    subperms = []
    # 使用 itertools.combinations 获取所有四元组
    for indices in itertools.combinations(range(len(perm)), length):
        # 提取对应位置的元素
        subperm = [perm[i] for i in indices]
        subperms.append(subperm)
    return subperms

# # 示例使用
# permutation = [1, 3, 2, 4, 5, 6]
# subpermutations = extract_subpermutations(permutation)

# print("Sub-permutations of length 4 with i < j < k < l:")
# for subperm in subpermutations:
#     print(subperm)



In [31]:
def ifsmooth(perm):
    """
    Check if a permutation is smooth.
    """
    subpermutations = extract_subpermutations(perm)
    for subperm in subpermutations:
        assert len(subperm) == 4, "Sub-permutations must have length 4"
        if subperm[1]>subperm[0] and subperm[0]>subperm[3] and subperm[3]>subperm[2]:
            return False
        if subperm[0]>subperm[2] and subperm[2]>subperm[1] and subperm[1]>subperm[3]:
            return False
    return True



In [32]:
if ifsmooth(permutation):
    print("The permutation is smooth.")
    S.add(tuple(permutation))
else:
    print("The permutation is not smooth.")
    N.add(tuple(permutation))

The permutation is not smooth.


In [33]:
def knuth_step(S, N):
    perm_list = set()
    for perm in N:
        perm = list(perm)  # Convert tuple to list
        for i in range(len(perm) - 2):
            # First kind of transformation
            if perm[i] > perm[i + 1] and perm[i] < perm[i + 2]:
                new_perm = perm.copy()
                new_perm[i+1], new_perm[i+2] = new_perm[i+2], new_perm[i+1]
                perm_list.add(tuple(new_perm))  # Store as tuple
            if perm[i] < perm[i + 1] and perm[i] > perm[i + 2]:
                new_perm = perm.copy()
                new_perm[i+1], new_perm[i+2] = new_perm[i+2], new_perm[i+1]
                perm_list.add(tuple(new_perm))

            # Second kind of transformation
            if perm[i+2] > perm[i] and perm[i+2] < perm[i+1]:
                new_perm = perm.copy()
                new_perm[i], new_perm[i+1] = new_perm[i+1], new_perm[i]
                perm_list.add(tuple(new_perm))
            if perm[i+2] < perm[i] and perm[i+2] > perm[i+1]:
                new_perm = perm.copy()
                new_perm[i], new_perm[i+1] = new_perm[i+1], new_perm[i]
                perm_list.add(tuple(new_perm))
    
    for perm in S:
        perm = list(perm)  # Convert tuple to list
        for i in range(len(perm) - 2):
            # First kind of transformation
            if perm[i] > perm[i + 1] and perm[i] < perm[i + 2]:
                new_perm = perm.copy()
                new_perm[i+1], new_perm[i+2] = new_perm[i+2], new_perm[i+1]
                perm_list.add(tuple(new_perm))  # Store as tuple
            if perm[i] < perm[i + 1] and perm[i] > perm[i + 2]:
                new_perm = perm.copy()
                new_perm[i+1], new_perm[i+2] = new_perm[i+2], new_perm[i+1]
                perm_list.add(tuple(new_perm))

            # Second kind of transformation
            if perm[i+2] > perm[i] and perm[i+2] < perm[i+1]:
                new_perm = perm.copy()
                new_perm[i], new_perm[i+1] = new_perm[i+1], new_perm[i]
                perm_list.add(tuple(new_perm))
            if perm[i+2] < perm[i] and perm[i+2] > perm[i+1]:
                new_perm = perm.copy()
                new_perm[i], new_perm[i+1] = new_perm[i+1], new_perm[i]
                perm_list.add(tuple(new_perm))
    
    return perm_list


In [34]:
while len(S) + len(N) < num_tableaux:
    for perm in knuth_step(S, N):
        perm_tuple = tuple(perm)  # 转为元组存储
        if ifsmooth(perm):
            S.add(perm_tuple)
        else:
            N.add(perm_tuple)
    assert len(S) + len(N) <= num_tableaux, "The number of permutations exceeds the number of standard Young tableaux."
print(f"Number of smooth permutations: {len(S)}")
print(f"Number of non-smooth permutations: {len(N)}")
    


Number of smooth permutations: 0
Number of non-smooth permutations: 15015


In [35]:
N

{(10, 3, 11, 6, 2, 1, 4, 5, 12, 13, 8, 7, 9),
 (10, 11, 3, 2, 12, 6, 4, 8, 5, 7, 13, 9, 1),
 (10, 3, 2, 11, 4, 6, 12, 5, 8, 7, 13, 1, 9),
 (10, 3, 11, 2, 4, 6, 5, 12, 8, 13, 1, 9, 7),
 (3, 10, 4, 11, 12, 2, 6, 1, 8, 5, 7, 13, 9),
 (10, 3, 4, 11, 12, 2, 6, 13, 8, 1, 5, 7, 9),
 (10, 11, 3, 4, 6, 12, 8, 2, 5, 1, 7, 13, 9),
 (3, 10, 4, 11, 2, 12, 13, 6, 5, 8, 9, 1, 7),
 (10, 3, 4, 2, 11, 6, 5, 12, 8, 13, 7, 1, 9),
 (10, 3, 2, 11, 6, 4, 5, 12, 1, 8, 7, 13, 9),
 (3, 10, 11, 12, 13, 4, 6, 2, 1, 8, 9, 5, 7),
 (10, 11, 3, 4, 6, 12, 2, 1, 13, 8, 9, 5, 7),
 (10, 11, 12, 3, 2, 4, 6, 5, 13, 8, 1, 9, 7),
 (10, 11, 3, 4, 12, 13, 6, 2, 1, 5, 8, 9, 7),
 (10, 3, 4, 11, 12, 13, 6, 2, 1, 8, 9, 5, 7),
 (10, 3, 11, 4, 2, 6, 12, 13, 8, 5, 1, 9, 7),
 (10, 11, 3, 12, 2, 1, 4, 13, 6, 8, 9, 5, 7),
 (10, 11, 3, 12, 2, 4, 6, 13, 5, 8, 9, 7, 1),
 (10, 11, 3, 12, 2, 13, 4, 6, 5, 8, 1, 9, 7),
 (3, 10, 4, 11, 2, 1, 6, 12, 13, 5, 8, 9, 7),
 (10, 11, 3, 2, 6, 4, 12, 5, 13, 8, 1, 7, 9),
 (10, 11, 12, 3, 2, 4, 6, 5, 1, 8,

In [36]:
S

set()